# Synthesize Intensity: Out-of-Phase Cooling with Non-equilibrium Ionization

In [1]:
import os
import time
import h5py
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
import numpy as np
import astropy.units as u
from astropy.visualization import SqrtStretch,ImageNormalize,AsinhStretch,LogStretch
import sunpy.sun.constants
import matplotlib.pyplot as plt
import matplotlib.colors
import dask
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.atomic import EmissionModel, Element
#from synthesizAR.interfaces import EbtelInterface
from synthesizAR.visualize import plot_aia_channels

%matplotlib inline

In [2]:
cluster = distributed.LocalCluster(n_workers=64,threads_per_worker=1)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:35028 Dashboard: http://127.0.0.1:41221/status,Cluster Workers: 64 Cores: 64 Memory: 270.38 GB


Load field and emission model

In [3]:
field = synthesizAR.Field.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/field_checkpoint/')

In [ ]:
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/base_emission_model.json')

## Population Fractions

In [ ]:
nei_futures = em_model.calculate_ionization_fraction(field,
                                                     '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/nei/ionization_fractions.h5',
                                                    interface=EbtelInterface)

In [ ]:
nei_futures.release()

In [ ]:
em_model.save('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/nei/emission_model.json')

In [ ]:
# Restore emission model here as needed
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/nei/emission_model.json')

## Flatten Detector Counts

In [4]:
aia = InstrumentSDOAIA([0,30000]*u.s,field.magnetogram.observer_coordinate)

/storage-home/w/wtb2/anaconda3/envs/synthesized-timelags/lib/python3.6/site-packages/sunpy-1.0.dev9869-py3.6-linux-x86_64.egg/sunpy/map/mapbase.py:645: Warning: Missing metadata for heliographic longitude: assuming longitude of 0 degrees
  lon=self.heliographic_longitude,


In [5]:
observer = synthesizAR.Observer(field, [aia], parallel=True)

In [6]:
observer.build_detector_files('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/nei/',
                              0.5 * u.Mm)

/storage-home/w/wtb2/anaconda3/envs/synthesized-timelags/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:299: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
/storage-home/w/wtb2/anaconda3/envs/synthesized-timelags/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:299: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))


In [ ]:
flatten_futures = observer.flatten_detector_counts(emission_model=em_model)

## Build Intensity Maps

In [7]:
bin_futures = observer.bin_detector_counts('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/nei/')

## Visualize

In [ ]:
plot_aia_channels(aia,15000 * u.s, '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling_outofphase_long/nei/',
                  norm=ImageNormalize(vmin=0,vmax=3e2,stretch=AsinhStretch()),
                  figsize=(15,8.975))